In [ ]:
import os
from langchain_community.utilities import SQLDatabase

# OpenAI API 키 설정 (이 예제에서는 사용되지 않음)
api_key = os.getenv("OPENAI_API_KEY")

# SQLite 데이터베이스 경로
db_path = "/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/Chinook.db"

# SQLite 데이터베이스 연결
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# 데이터베이스 방언 출력 (예: sqlite)
print(db.dialect)

# 사용 가능한 테이블 이름 출력
print(db.get_usable_table_names())

# SQL 쿼리 실행 및 결과 출력
result = db.run("SELECT * FROM Artist LIMIT 10;")
print(result)

체인
<br>질문을 받아 SQL 쿼리로 변환하고, 쿼리를 실행한 후, 결과를 사용하여 원래 질문에 답변하는 간단한 체인을 만들어보겠습니다.

질문을 SQL 쿼리로 변환하기
SQL 체인이나 에이전트의 첫 번째 단계는 사용자 입력을 받아 SQL 쿼리로 변환하는 것입니다. LangChain에는 이를 위한 내장 체인이 있습니다: create_sql_query_chain.

In [ ]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "안녕"})
print(response)
print(db.run(response))

In [ ]:
chain.get_prompts()[0].pretty_print()

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

In [85]:
import pprint
from operator import itemgetter
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# OpenAI LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# SQL 쿼리 체인 생성
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

# PromptTemplate 설정
answer_prompt = PromptTemplate.from_template(
    """다음 사용자 질문, 해당하는 SQL 쿼리, 그리고 SQL 결과를 바탕으로 사용자 질문에 한국어로 답변해 주세요.

질문: {question}
SQL 쿼리: {query}
SQL 결과: {result}
답변: """
)


# answer_prompt = PromptTemplate.from_template(
#     """
# 사용자 질문에 친절하게 답변해 주세요.
# 데이터베이스와 관련된 질문인 경우에는 사용자 질문, 해당하는 SQL 쿼리, 그리고 SQL 결과를 바탕으로 사용자 질문에 한국어로 답변해 주세요.
# 질문: {question}

# 답변: """
# )


# StrOutputParser 설정
output_parser = StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt | llm | output_parser
)


response = chain.invoke({"question": "대한민국의 수도는?"})
generated_sql = write_query.invoke({"question": "대한민국의 수도는?"})
sql_result = db.run(generated_sql)

print("1. 생성된 SQL Query :")
print(generated_sql)
print("\n")

print("2. SQL Query 실행 결과 :")
print(sql_result)
print("\n")

print("3. 최종 답변 :")
pprint.pprint(response)
print("\n")


1. 생성된 SQL Query :
SELECT "City" 
FROM "Customer" 
WHERE "Country" = 'South Korea' 
LIMIT 1;


2. SQL Query 실행 결과 :



3. 최종 답변 :
('대한민국의 수도는 서울입니다. SQL 쿼리에 오류가 있어 정확한 결과를 얻을 수 없었습니다. 서울은 대한민국의 수도이며, 해당 정보는 '
 '데이터베이스에 저장되어 있습니다.')




In [86]:
import pprint
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate

# OpenAI LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 데이터베이스 쿼리 도구 설정
tools = [QuerySQLDataBaseTool(db=db, description="사용자의 질문을 데이터베이스 쿼리로 만들어 실행할 때만 사용하세요.")]
tool_names = [tool.name for tool in tools]

# PromptTemplate 설정
answer_prompt = PromptTemplate.from_template(
    """ 
사용자 질문에 친절하게 답변해 주세요.
만약, 데이터베이스와 관련된 질문인 경우에는 {tool_names} 중 하나를 사용하여 사용자 질문, 해당하는 SQL 쿼리, 그리고 SQL 결과를 바탕으로 사용자 질문에 한국어로 답변해 주세요.

질문: {question}
SQL 쿼리: {query}
SQL 결과: {result}
답변: """
)


# 에이전트 생성
agent_executor = create_sql_agent(
    llm, 
    db=db, 
    tools=tools,  # 데이터베이스 쿼리용 도구 추가
    agent_type="openai-tools", 
    verbose=True
)

# 사용자 질문 설정
user_question = "대한민국의 수도는?"

formatted_prompt = answer_prompt.format(
    question=user_question,
    tools=tools,
    tool_names=', '.join(tool_names),
    query="{query}",  # Placeholder for query
    result="{result}"  # Placeholder for result
)

response = agent_executor.invoke({"input": formatted_prompt})

print("Agent 답변 :")
pprint.pprint(response)




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Employee'}`



CREATE TABLE "Employee" (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"ReportsTo" INTEGER, 
	"BirthDate" DATETIME, 
	"HireDate" DATETIME, 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60), 
	PRIMARY KEY ("EmployeeId"), 
	FOREIGN KEY("ReportsTo") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Employee table:
EmployeeId	LastName	FirstName	Title	ReportsTo	BirthDate	HireDate	Address	City	State	Country	PostalCode	Phone	Fax	Email
1	Adams	Andrew	General Manager	None	1962-02-18 00:00:00	2002-08-14 00:0


ChatGPT
에이전트
LangChain에는 SQL 데이터베이스와 상호 작용하는 더 유연한 방법을 제공하는 SQL 에이전트가 있습니다. SQL 에이전트를 사용하면 다음과 같은 주요 장점이 있습니다:

데이터베이스의 스키마와 내용(예: 특정 테이블 설명)에 기반하여 질문에 답할 수 있습니다.
생성된 쿼리를 실행하고, 오류를 포착하여 올바르게 재생성함으로써 오류를 복구할 수 있습니다.
여러 종속 쿼리가 필요한 질문에 답할 수 있습니다.
관련 테이블의 스키마만 고려하여 토큰을 절약할 수 있습니다.
에이전트를 초기화하려면 create_sql_agent 함수를 사용합니다. 이 에이전트는 쿼리 생성 및 실행, 쿼리 구문 검사, 테이블 설명 검색 등을 수행할 수 있는 SQLDatabaseToolkit을 포함하고 있습니다.

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)

In [ ]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})